## COMCATE code enforcement violation summary  
Accepts standard report of annual code violations, by violation event number keyed by apn ('acctid').  This report may have multiple voilations per event, and this notebook splits those out to be counted.

Scoring the violations will require adding a table mapping violation type to scoring weight.  The current verions just counts all the violations up, as equal.

The bottom of this sheet clusters the violations by point of contact, and writes that out to a file for later use in group calculations.


In [1]:
!pip install leven
!pip install simpledbf
!pip install ngram

In [2]:
import pandas as pd
import re
from simpledbf import Dbf5
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Load the sdat property dataset

This data is used when trying to fix the code violation to add missing tax ids

In [4]:
sdat_columns = ['acctid',
       'ownname1','ownname2', 'namekey', 'ownadd1', 'ownadd2', 
       'owncity', 'ownstate', 'ownerzip', 'ownzip2']
rental_columns = ['Business Name', 'License Id', 'address',
       'Customer Id', 'Dist/Account No', 'RENTAL']
rental_merge_columns= sdat_columns + rental_columns
violation_merge_columns = ['address']#,'Open_Date','Follow Up Date']

### Enrich the billing data with columns from prior year(s), to get started

In [5]:
cambridge_apns = pd.read_csv('/content/drive/My Drive/SDAT/CAN-ref.csv')
cambridge_apns = cambridge_apns[~cambridge_apns["acctid"].str.contains("-")]
cambridge_apns.acctid = cambridge_apns.acctid.astype(int)

In [6]:
sdat = pd.read_csv('drive/My Drive/pita 2021/SDAT-CAN-ref-202105.csv')
sdat.acctid = sdat.acctid.astype(int)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,29,38,45,49,52,53,77,82,86,104,106,112,113,129,137) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def format_property_location(x):
  return re.sub(r"(^.*)( [ \d]+\w?)$", r"\1", x.strip().upper())

rentals = pd.read_csv('/content/drive/My Drive/pita 2021/cleaned_rental_billing-2020-test.csv')
rentals['address'] = rentals['Property Location'].apply(lambda x: format_property_location(x))
rentals.address = rentals.apply(lambda x: re.sub(r' {2,}', ' ', str(x.address)),axis=1)
rentals.loc[rentals.address == "738 BAYLY RD", 'acctid'] = 1007169094
#rentals.loc[rentals.address == "501 ACADEMY ST APT 2", 'acctid'] = 1007169094
rentals.loc[rentals.address == "310 DORCHESTER AVE DOWN", 'address'] = 1007139101
rentals.loc[rentals.address == "411 EAGLES NEST WAY(#69)7304", 'acctid'] = 1007219164

print("base:",len(rentals))
rentals.drop_duplicates(subset=['acctid','Customer Id','address'],keep='last',inplace=True)
print("removing mutiple licenses per unique acctid-address:",len(rentals))
rentals = rentals.merge(sdat[sdat_columns],on='acctid',how='outer',indicator=True)
print(len(rentals.query('_merge == "both"')))
temp = rentals.query('_merge == "left_only"')[rental_columns].merge(sdat,on='address',how='outer',indicator=True)
rentals = pd.concat([rentals.query('_merge =="both"')[rental_merge_columns],temp.query('_merge == "both" and city == "CAMBRIDGE"')[rental_merge_columns]])
print(len(rentals))
temp.query('_merge == "left_only"')

base: 1534
removing mutiple licenses per unique acctid-address: 1529
1523
1527


,Business Name,License Id,address,Customer Id,Dist/Account No,RENTAL,acctid,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,...,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address,_merge
2,HAMBLETON CREEK PROP LLC,19-00535,501 ACADEMY ST APT 2,RR-03828,07-146310,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,MAIN STREET HOUSING INC,19-00973,310 DORCHESTER AVE DOWN,RR-08003,07-169101,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


#### Format violations list dates and violations lists
pandas df.explode() is used to make records for each row with seprate violations, but we won't do that until needed to avoid memeory pressure when clustering.

In [8]:
violations = pd.read_csv("drive/My Drive/comcate/code_violations-20210701.csv")
violations['Open Date'] = violations.apply(lambda x: pd.to_datetime(x['Open Date']), axis = 1)
violations = violations.rename(columns={'Open Date':'Open_Date'})

In [9]:
import re
print(len(violations))
violations.address = violations.apply(lambda x: re.sub(r' {2,}', ' ', str(x.address)),axis=1)

3673


#### fixup account ids

In [10]:
violations.drop_duplicates(inplace=True)
violations.APN = violations.APN.fillna(0).astype(int)
violations.APN = violations.APN.astype(str)
violations = violations.query('APN.str.contains("[0-9]+")',engine='python')

In [11]:
#violations.merge(rentals.rename(columns={'acctid':'APN'})[['APN','License Id','address']],on=['APN','address'],how='outer',indicator=True).query("_merge =='left_only'")

In [12]:
import re

def split_violations(x):
  temp = re.sub(r'[-\n:]+','',x).lower().strip()
  temp = re.split(r'(closed|open)',temp)
  return [x.strip() for x in temp if not 'closed' in x and not 'open' in x and x.strip() != '']

print(len(violations))
violations['vtypes'] = violations.apply(lambda x: split_violations(x['Violation(s)']),axis=1)

v_flat = violations.explode('vtypes')
len(v_flat)

3673


6502

In [13]:
v_categories = pd.read_csv('/content/drive/My Drive/pita 2021/violation_types.csv')
v_categories.violation_types = v_categories.violation_types.apply(lambda x: x.lower())
v_lookup = dict(zip(v_categories.violation_types,v_categories.category))
v_flat['violation_category_text'] = v_flat.vtypes.apply(lambda x: x.split('-')[0].strip())

def lookup_category(x):
  if x in v_lookup.keys():
    return v_lookup[x]
  else:
    return 0

v_flat['type_0'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) < 1 else 0,axis=1)
v_flat['type_1'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 1 else 0,axis=1)
v_flat['type_2'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 2 else 0,axis=1)
v_flat['type_3'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 3 else 0,axis=1)
v_flat['type_4'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 4 else 0,axis=1)

#### Summarize violations of all types.
Split the multi-line violation descriptions, and total the number of individual issues per tax account id.

In [14]:
len(v_flat)#.query('APN == 1007127049')

6502

In [15]:
import numpy as np
ht_score = v_flat.groupby(['APN','address']).agg({'type_0':['sum'],'type_1':['sum'],'type_2':['sum'],'type_3':['sum'],'type_4':['sum']})
ht_score.columns = ['_'.join(col).strip() for col in ht_score.columns.values]
#ht_score

In [16]:
len(ht_score)

1448

In [17]:
housing_quality_tier_scores = [0,6,5,1,0.5,10]

lo_bldg_by_tier = {1:80,2:85,3:100}
hi_bldg_by_tier = {1:140,2:165,3:195}
lo_unit_by_tier = {1:30,2:60,3:160}
hi_unit_by_tier = {1:10,2:20,3:80}
building_fees_by_capacity = {'low': lo_bldg_by_tier,'high': hi_bldg_by_tier}
unit_fees_by_capacity = {'low': lo_unit_by_tier ,'high': hi_unit_by_tier}
supplemental_fee_by_tier = {'low':{1:0,2:100,3:205},'high':{1:0,2:155,3:410}}

def pc_score(x): 
  return x.type_1_sum * housing_quality_tier_scores[1] + \
  x.type_2_sum * housing_quality_tier_scores[2] + \
  x.type_3_sum * housing_quality_tier_scores[3] + \
  x.type_4_sum * housing_quality_tier_scores[4]

def pc_tier(x):
  if x > 39:
    return 3
  elif x > 19:
    return 2
  else:
    return 1

def pc_fee(tier,units):
  fee = 0
  per_unit_fee = 0
  building_fee = 0
  capacity_fee = 0
  sup_fee = 0

  if units > 3:
    building_fee = building_fees_by_capacity['high'][tier]
    per_unit_fee = unit_fees_by_capacity['high'][tier]
    capacity_fee = per_unit_fee * units
    sup_fee = supplemental_fee_by_tier['high'][tier]
  else:
    building_fee = building_fees_by_capacity['low'][tier]
    per_unit_fee = unit_fees_by_capacity['low'][tier]
    capacity_fee = per_unit_fee * units
    sup_fee = supplemental_fee_by_tier['low'][tier]
  fee = building_fee + capacity_fee + sup_fee
  if tier  > 1:
    print(tier,units,per_unit_fee,building_fee,'+',capacity_fee,'+',sup_fee,'=',fee,'vs',units*56)
  return fee

ht_score['property_condition_score'] = ht_score.apply(lambda x: pc_score(x),axis=1)
#ht_score['property_condition_tier'] = ht_score.apply(lambda x: pc_tier(x.property_condition_score),axis=1)
ht_score = ht_score.reset_index()
ht_score.rename(columns={'APN':'acctid'},inplace=True)
ht_score.acctid = ht_score.acctid.astype(int)

ht_score[ht_score.property_condition_score > 40]

,acctid,address,type_0_sum,type_1_sum,type_2_sum,type_3_sum,type_4_sum,property_condition_score
18,1007104863,405 HENRY ST,2,2,5,11,11,53.5
72,1007107730,832 WASHINGTON ST,2,4,5,9,16,66.0
154,1007113358,813 RACE ST,2,7,5,10,11,82.5
191,1007117345,802 HIGH ST,0,5,3,6,10,56.0
282,1007122047,802 WASHINGTON ST,0,0,8,0,16,48.0
291,1007122195,1016 PINE ST,0,6,0,1,11,42.5
315,1007123655,521 PINE ST,1,5,2,6,5,48.5
371,1007126034,909 MACES LANE,1,2,5,7,18,53.0
374,1007126166,605 DOUGLAS ST,2,5,6,16,2,77.0
384,1007127049,509 EDGEWOOD AVE,2,10,16,25,17,173.5


In [18]:
print(len(ht_score))
len(ht_score.acctid.unique())

1448


1431

In [19]:
results = ht_score.drop(columns='address').merge(rentals,on='acctid',how='right')
results.columns

aggregation_map = dict(zip(results.columns,['first' for k in range(len(results.columns))]))
for k in ['type_0_sum','type_1_sum', 'type_2_sum', 'type_3_sum','type_4_sum','property_condition_score']:
  aggregation_map[k] = 'sum'

In [20]:
#results = ht_score.drop(columns='address').merge(rentals,on='acctid',how='right')
#results.property_condition_tier = results.property_condition_tier.fillna(1)
results = results.fillna(0)
print(len(results))
results = results.merge(cambridge_apns,on='acctid',how='inner')

# THE FIX!!
results = results.groupby(['acctid','License Id']).agg(aggregation_map)
results['property_condition_tier'] = results.apply(lambda x: pc_tier(x.property_condition_score),axis=1)
results = results.drop(columns=['acctid','License Id']).reset_index()


1536


In [22]:
results['fee'] = results.apply(lambda x: pc_fee(x.property_condition_tier,x['RENTAL']),axis=1)
results['std_fee'] = results.apply(lambda x: x['RENTAL'] * 56,axis=1)
results.accitid = results.acctid.astype(int)
results.drop_duplicates(inplace=True)
results[results.property_condition_tier > 2]


2 2.0 60 85 + 120.0 + 100 = 305.0 vs 112.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 4.0 20 165 + 80.0 + 155 = 400.0 vs 224.0
2 2.0 60 85 + 120.0 + 100 = 305.0 vs 112.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
3 2.0 160 100 + 320.0 + 205 = 625.0 vs 112.0
3 2.0 160 100 + 320.0 + 205 = 625.0 vs 112.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 2.0 60 85 + 120.0 + 100 = 305.0 vs 112.0
3 2.0 160 100 + 320.0 + 205 = 625.0 vs 112.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
3 2.0 160 100 + 320.0 + 205 = 625.0 vs 112.0
3 121.0 80 195 + 9680.0 + 410 = 10285.0 vs 6776.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


,acctid,License Id,type_0_sum,type_1_sum,type_2_sum,type_3_sum,type_4_sum,property_condition_score,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,Business Name,address,Customer Id,Dist/Account No,RENTAL,property_condition_tier,fee,std_fee
264,1007122047,19-01349,0.0,0.0,8.0,0.0,16.0,48.0,CWJ LLP,0,CWJ LLP,1240 PENNSYLVANIA AVE,0,MADISON,MD,21648.0,0.0,CWJ RENTALS,802 WASHINGTON ST,RR-03023,07-122047,1.0,3,465.0,56.0
338,1007126166,19-01364,2.0,5.0,6.0,16.0,2.0,77.0,CORNISH ALFONSO N ETAL,AUSTIN CORNISH YVONNE DENISE,CORNISH ALFONSO N ETAL,10241 DORSEY POINTE CIR,0,LOUISVILLE,KY,40223.0,0.0,CORNISH ALFONSO,605 DOUGLAS ST,RR-03631,0,1.0,3,465.0,56.0
350,1007127049,19-00307,3.0,11.0,18.0,29.0,19.0,194.5,GLG HOLDINGS LLC,0,GLG HOLDINGS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,0.0,GLG HOLDINGS LLC,509 EDGEWOOD AVE,RR-03214,07-127049,2.0,3,625.0,112.0
351,1007127049,19-00308,3.0,11.0,18.0,29.0,19.0,194.5,GLG HOLDINGS LLC,0,GLG HOLDINGS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,0.0,GLG HOLDINGS LLC,517 EDGEWOOD AVE,RR-03214,07-127049,2.0,3,625.0,112.0
352,1007127073,19-00682,1.0,5.0,2.0,7.0,3.0,48.5,GLG HOLDINGS LLC,0,GLG HOLDINGS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,0.0,GLG HOLDINGS LLC,808 PHILLIPS ST,RR-03214,07-127073,1.0,3,465.0,56.0
354,1007127138,19-00775,11.0,3.0,2.0,14.0,2.0,43.0,MADISON BAY PROPERTIES LLC,0,MADISON BAY PROPERTIES LL,1240 PENNSYLVANIA AVE,0,MADISON,MD,21648.0,1105.0,ED JAMES,857 PARK LANE,RR-03136,07-127138,1.0,3,465.0,56.0
367,1007127529,19-00342,1.0,2.0,6.0,8.0,5.0,52.5,CHEROKEE RENTALS LLC,0,CHEROKEE RENTALS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,0.0,CHEROKEE RENTALS LLC,715 DOUGLAS ST 459-B,RR-03465,07-127529,2.0,3,625.0,112.0
370,1007127553,19-00360,0.0,9.0,2.0,0.0,2.0,65.0,CHEROKEE RENTALS LLC,0,CHEROKEE RENTALS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,0.0,CHEROKEE RENTALS LLC,703 DOUGLAS ST,RR-03620,07-127553,2.0,3,625.0,112.0
403,1007129688,19-00397,0.0,6.0,10.0,80.0,10.0,171.0,BRADFORD HOUSE ASSOCIATES LLC,0,BRADFORD HOUSE ASSOCIATES,C/O PRESERVATION MGMT INC,261 GORHAM RD,SOUTH PORTLAND,ME,4106.0,0.0,BRADFORD HOUSE ASSOCIATES LLC,701 RACE ST,RR-03927,07-129688,121.0,3,10285.0,6776.0
557,1007139438,19-00092,0.0,0.0,7.0,2.0,9.0,41.5,OTTER LLC,0,OTTER LLC,PO BOX 1181,0,CAMBRIDGE,MD,21613.0,0.0,OTTER LLC,201 PHILLIPS AVE,RR-00782,07-139438,1.0,3,465.0,56.0


In [23]:
results.to_csv('/content/drive/My Drive/pita 2021/property_tiering-20210707.csv')

THERE ARE PROBLEMS IN THE RENTAL DATA SET THAT DROPS RECORDS HERE

In [ ]:
# violations['vtypes'] = violations.apply(lambda x: x['Violation(s)'].split('\n'),axis=1)
# v_flat = violations.explode('vtypes')
vsummary = v_flat.groupby('APN')['Violation(s)'].value_counts().reset_index(name='vcount')
vs2 = vsummary.groupby(['APN'])['vcount'].sum().reset_index(name='violations_total')
vs2


NameError: ignored

In [ ]:
#rentals.query('acctid == "1007229100"')

In [ ]:

import re
!cp "drive/My Drive/pita 2021/text_cluster.py" .
from text_cluster import assign_clusters
from text_cluster import text_similarity
from text_cluster import cluster_strings

def format_contact(s):
  print(s)
  cinfo = re.sub(r'[,.;:-]','', s).split("\n")
  name = cinfo[0]
  address = ""
  print(cinfo)
  if "Property Owner" in cinfo[0]:
    if len(cinfo) > 2:
      name = cinfo[1].replace("'","").upper()
      address = re.sub(r',* *[A-Z][A-Z] \d\d\d\d\d\'*$',"",cinfo[2]).replace("'","").upper()
    else:
      match = re.search(r' \d+ ',cinfo[1])
      if match:
        tokens = cinfo[1].upper().split(" ")
        numbers = [i for i,token in enumerate(tokens) if re.search(r'\d',token)]
        if len(numbers) > 0:
          name = " ".join(tokens[0:numbers[0]])
          address = " ".join(tokens[numbers[0]:-1])
  else:
    print("BAD RECORD ",s)
  return pd.Series([name, address], index=['Contact_Name','Contact_Address'])


violations[['Contact_Name','Contact_Address']] = violations['Contacts'].apply(format_contact)
violations['address'] = violations.fillna("").apply(lambda x: " ".join([str(x['Street Number']),str(x['Street Direction']),str(x['Street Name']),str(x['Street Type'])]).upper(),axis=1)
violations.APN = violations.apply(lambda x: x.APN.strip(u'\u200b'),axis=1)
violations.address = violations.apply(lambda x: re.sub(r' +',' ',x.address).strip(),axis=1)

####Cluster the violations
Assign clusters by contact name and contact address, split from the Contact column.

In [ ]:
%%time
myquery = violations.query('(Open_Date > "2018-12-31")',engine='python')
names = cluster_strings(myquery.Contact_Name.array,'c_contact',metric='jaccard',threshold=0.1)
addresses = cluster_strings(myquery.Contact_Address.array,'c_address',metric='jaccard',threshold=0.1)

In [ ]:
violations['contact_cluster'] = violations.apply(lambda x: names.get(x.Contact_Name,-1), axis=1)
violations['contactadd_cluster'] = violations.apply(lambda x: addresses.get(x.Contact_Address,-1), axis=1)

#### At this point we have clusters for all the code violations

Some tax accounts are associated with more than one cluster.  This can be due to entry errors in the complaint data, or changes in ownership over the complaint period. The default behavior, used as the fixed behavior here, is to return the least ambiguous dominant cluster id.  What this means practically is if the most often cited cluster == -1 (no cluster), but the second most frequent citation in the period is a valid cluster, that valid cluster is assinged to the tax id.  If more than one valid cluster is cited, the most frequently cited is used.

Some of the violations have no tax acctid, in some cases because of address spelling issues.  Some of these can be backfilled.  The ones we can fix, we do.  Many of the others are just vacant lots which will drop out on the join with properties licenesed for rental.

In [ ]:
def best_cluster(cnumbers):
  if len(cnumbers) > 1 and cnumbers.index[0] < 0:
    return cnumbers.index[1]
  else:
    return cnumbers.index[0]
    
vc_df = pd.DataFrame(violations.groupby(['APN'])['contactadd_cluster'].agg(lambda x:best_cluster(x.value_counts())))#.to_dict()
cadd_c = violations.groupby(['APN'])['contact_cluster'].agg(lambda x:best_cluster(x.value_counts()))
vc_df = vc_df.merge(cadd_c,left_index=True,right_index=True)

# fix tax account id's where possible
vc_df = vc_df.merge(vs2, on='APN',how='inner')
vc_df

### Save result as violation_clusters.csv


In [ ]:
vc_df.to_csv('drive/My Drive/pita 2020/violation_clusters.csv')

In [ ]:
vsample = violations.query('(Open_Date > "2018-12-31")')[['APN','address','contact_cluster','Contact_Name','contactadd_cluster','Contact_Address']]
vsample.APN = vsample.APN.astype(str)
vsample = vsample.rename(columns={'APN':'acctid'})
vsample.acctid = vsample.apply(lambda x: x.acctid.strip(u'\u200b'),axis=1)
vsample.address = vsample.apply(lambda x: re.sub(r' +',' ',x.address).strip(),axis=1)

noacctid = vsample.query('acctid == ""',engine='python').address.array
found_acctids = {v[0]:v[1] for v in df.query('address in @noacctid')[['address','acctid']].values}
vsample.acctid = vsample.apply(lambda x: found_acctids.get(x.address,x.acctid),axis=1)
vsample.query('acctid == ""',engine='python').address.array
#.query('Contact_Name.str.contains("GLG")',engine='python')